# Cloud Workshop Microsoft
## 3. Machine Learning

In [2]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.21

In [4]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /databricks/driver/aml_config/config.json
Workspace name: azuremlservice
Azure region: westeurope
Resource group: azuremlserviceresourcegroup

In [5]:
#get the train and test datasets
train_data_path = "AdultCensusIncomeTrain"
test_data_path = "AdultCensusIncomeTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("train: ({}, {})".format(train.count(), len(train.columns)))
print("test: ({}, {})".format(test.count(), len(test.columns)))

train.printSchema()

train: (25967, 15)
test: (6594, 15)
root
-- education: string (nullable = true)
-- marital_status: string (nullable = true)
-- age: integer (nullable = true)
-- workclass: string (nullable = true)
-- capital_gain: integer (nullable = true)
-- sex: string (nullable = true)
-- capital_loss: integer (nullable = true)
-- fnlwgt: integer (nullable = true)
-- hours_per_week: integer (nullable = true)
-- native_country: string (nullable = true)
-- relationship: string (nullable = true)
-- education_num: integer (nullable = true)
-- occupation: string (nullable = true)
-- race: string (nullable = true)
-- income: string (nullable = true)

#Define Model

In [7]:
label = "income"
dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip"))
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

# string-index the label column into a column named "label"
si_label = StringIndexer(inputCol=label, outputCol='label')

# assemble the encoded feature columns in to a column named "features"
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

In [8]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import numpy as np
import os
import shutil

model_name = "AdultCensus_runHistory.mml"
model_dbfs = os.path.join("/dbfs", model_name)
run_history_name = 'spark-ml-notebook'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "Ignite_AI_Talk")
root_run = myexperiment.start_logging()

# Regularization Rates - 
regs = [0.0001, 0.001, 0.01, 0.1]

# try a bunch of regularization rate in a Logistic Regression model
for reg in regs:
    print("Regularization rate: {}".format(reg))
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        # create a new Logistic Regression model.
        lr = LogisticRegression(regParam=reg)
        
        # put together the pipeline
        pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

        # train the model
        model_p = pipe.fit(train)
        
        # make prediction
        pred = model_p.transform(test)
        
        # evaluate. note only 2 metrics are supported out of the box by Spark ML.
        bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
        au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
        au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

        print("Area under ROC: {}".format(au_roc))
        print("Area Under PR: {}".format(au_prc))
      
        # log reg, au_roc, au_prc and feature names in run history
        run.log("reg", reg)
        run.log("au_roc", au_roc)
        run.log("au_prc", au_prc)
        run.log_list("columns", train.columns)

        # save model
        model_p.write().overwrite().save(model_name)
        
        # upload the serialized model into run history record
        mdl, ext = model_name.split(".")
        model_zip = mdl + ".zip"
        shutil.make_archive(mdl, 'zip', model_dbfs)
        run.upload_file("outputs/" + model_name, model_zip)        
        #run.upload_file("outputs/" + model_name, path_or_stream = model_dbfs) #cannot deal with folders

        # now delete the serialized model from local folder since it is already uploaded to run history 
        shutil.rmtree(model_dbfs)
        os.remove(model_zip)
        
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Regularization rate: 0.0001
Area under ROC: 0.9062833374559738
Area Under PR: 0.7729351517367892
Regularization rate: 0.001
Area under ROC: 0.9062612140405663
Area Under PR: 0.7729827472628713
Regularization rate: 0.01
Area under ROC: 0.9033025509548078
Area Under PR: 0.7644985083289214
Regularization rate: 0.1
Area under ROC: 0.8934131342842805
Area Under PR: 0.7411948490196643
run id: 2a842f49-9bfa-45c8-828d-a6d0436d2717

In [9]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}

for r in root_run.get_children():
    child_runs[r.id] = r

In [10]:
metrics = root_run.get_metrics(recursive=True)
best_run_id = max(metrics, key = lambda k: metrics[k]['au_roc'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', metrics[best_run_id]['au_roc'], metrics[best_run_id]['reg'])

Best run is: 96b6c59a-594e-4e5a-802e-383bbfc0d5f6
Metrics: 0.9062833374559738 0.0001

In [11]:
#Download the model from the best run to a local folder
best_model_file_name = "best_model.zip"
best_run.download_file(name = 'outputs/' + model_name, output_file_path = best_model_file_name)

#Model Evaluation

In [13]:
##unzip the model to dbfs (as load() seems to require that) and load it.
if os.path.isfile(model_dbfs) or os.path.isdir(model_dbfs):
    shutil.rmtree(model_dbfs)
shutil.unpack_archive(best_model_file_name, model_dbfs)

model_p_best = PipelineModel.load(model_name)

In [14]:
# make prediction
pred = model_p_best.transform(test)
output = pred[['hours_per_week','age','marital_status','income','prediction']]
display(output.limit(8))

hours_per_week,age,marital_status,income,prediction
40,20,Divorced,<=50K,0.0
30,26,Divorced,<=50K,0.0
40,31,Divorced,<=50K,0.0
40,32,Divorced,<=50K,0.0
35,33,Divorced,<=50K,0.0
38,34,Divorced,<=50K,0.0
40,37,Divorced,<=50K,0.0
48,41,Divorced,<=50K,0.0


In [15]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.9062833374559738
Area Under PR: 0.7729351517367892

#Model Persistence

In [17]:
##NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_p_best.write().overwrite().save(model_name)
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/AdultCensus_runHistory.mml

In [18]:
%sh

ls -la /dbfs/AdultCensus_runHistory.mml/*

/dbfs/AdultCensus_runHistory.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Apr 3 07:55 .
drwxr-xr-x 1 root root 0 Apr 3 07:55 ..
-rw-r--r-- 1 root root 736 Apr 3 07:55 part-00000
-rw-r--r-- 1 root root 0 Apr 3 07:55 _SUCCESS

/dbfs/AdultCensus_runHistory.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Apr 3 07:55 .
drwxr-xr-x 1 root root 0 Apr 3 07:55 ..
drwxr-xr-x 1 root root 0 Apr 3 07:55 00_StringIndexer_797957b480e6
drwxr-xr-x 1 root root 0 Apr 3 07:55 01_StringIndexer_c2e388573e11
drwxr-xr-x 1 root root 0 Apr 3 07:55 02_StringIndexer_579589d757ed
drwxr-xr-x 1 root root 0 Apr 3 07:55 03_StringIndexer_40e43fb64398
drwxr-xr-x 1 root root 0 Apr 3 07:55 04_StringIndexer_39a4fdca1204
drwxr-xr-x 1 root root 0 Apr 3 07:55 05_StringIndexer_62559c59e5d6
drwxr-xr-x 1 root root 0 Apr 3 07:55 06_StringIndexer_81316b481137
drwxr-xr-x 1 root root 0 Apr 3 07:55 07_StringIndexer_7ecb5a690d0e
drwxr-xr-x 1 root root 0 Apr 3 07:55 08_OneHotEncoder_cd7c2cc30f39
drwxr-xr-x 1 root root 0 Apr 3 07:55 09_OneHotEncoder_a6f6d0986983
drwxr-xr-x 1 root root 0 Apr 3 07:55 10_OneHotEncoder_8a141db592fc
drwxr-xr-x 1 root root 0 Apr 3 07:55 11_OneHotEncoder_4dfdfb2f0fe1
drwxr-xr-x 1 root root 0 Apr 3 07:55 12_OneHotEncoder_80f31ef1482d
drwxr-xr-x 1 root root 0 Apr 3 07:55 13_OneHotEncoder_577fd1983834
drwxr-xr-x 1 root root 0 Apr 3 07:55 14_OneHotEncoder_951c25eda29e
drwxr-xr-x 1 root root 0 Apr 3 07:55 15_OneHotEncoder_6fd00dfe72d3
drwxr-xr-x 1 root root 0 Apr 3 07:55 16_StringIndexer_a1219315286e
drwxr-xr-x 1 root root 0 Apr 3 07:55 17_VectorAssembler_8240b0504e7a
drwxr-xr-x 1 root root 0 Apr 3 07:55 18_LogisticRegression_4c98ed8b7540

In [19]:
dbutils.notebook.exit("success")

success